In [1]:
#!pip install dgl-cu102 hyperopt ase 

In [2]:
import os
os.environ['DGLBACKEND'] = "pytorch"

In [3]:
import numpy as np

from network import CG_CNN_Layer, Net
from dataset import GraphDataset

import dgl
from dgl.data.utils import split_dataset

from time import perf_counter

import torch as th

from datetime import datetime
from multiprocessing import cpu_count

import warnings

Using backend: pytorch


In [4]:
warnings.filterwarnings("ignore", "Recommend creating graphs by")

In [5]:
# CUDA for PyTorch
use_cuda = th.cuda.is_available()
device = th.device("cuda:0" if use_cuda else "cpu")
# th.backends.cudnn.benchmark = True

In [21]:
train_idxs, valid_idxs, test_idxs = split_dataset(dataset = np.arange(1,2401), 
                                                  frac_list=[0.7,0.2,0.1], shuffle=True, 
                                                  random_state=42)

trainingset = GraphDataset("data.bin", tuple(train_idxs))
validset = GraphDataset("data.bin", tuple(valid_idxs))

In [7]:
obj_params = {'architecture': {'activation': th.nn.modules.activation.ReLU,
  'n_conv': 1,
  'neuron_ratios': ((3, 2), (200, 100))},
 'batch_size': 32,
 'optimizer_params': {'amsgrad': False,
  'betas': (0.9, 0.9999),
  'lr': 0.01,
  'weight_decay': 0.0025}}

In [8]:
obj_params

{'architecture': {'activation': torch.nn.modules.activation.ReLU,
  'n_conv': 1,
  'neuron_ratios': ((3, 2), (200, 100))},
 'batch_size': 32,
 'optimizer_params': {'amsgrad': False,
  'betas': (0.9, 0.9999),
  'lr': 0.01,
  'weight_decay': 0.0025}}

In [11]:
in_feats = 4
net = Net(in_feats=in_feats, **obj_params['architecture']).to(device)
opt = th.optim.Adam(net.parameters(), **obj_params['optimizer_params'])
loss = th.nn.MSELoss()
v_loss = th.nn.MSELoss()
max_epochs = 300
verbose = True
nan_count = []
min_ep_v_loss = 100
epoch_time = []
nan_batches = 0

In [12]:
for epoch in range(20):
    t_start = perf_counter()
    ep_t_loss = 0
    ep_v_loss = 0
    j=0
    #with th.autograd.detect_anomaly():
    for graph, label, index in trainingset:
        j+=1
        label = label['glabel'][index]
        graph = graph[0]
        # Transfer to GPU
        graph, label = graph.to(
            device), label.to(device)

        # Model computations
        pred = net(graph)
#         try:
#             pred = net(local_batch)
#         except KeyError:
#             j-=1
#             nan_batches+=1
#             opt.zero_grad()
#             continue
#             print(pred.shape)
#             print(local_targets.shape)
        ep_t_loss+=loss(pred.float(), label.float())
    ep_t_loss/=j
    ep_t_loss.backward()
    opt.step()
    opt.zero_grad()
    t_end = perf_counter()
    
#     j=0
#     for graph, label, index in validset:
#         j+=1

#         label = label['glabel'][index]
#         graph = graph[0]
#         # Transfer to GPU
#         graph, label = graph.to(
#             device), label.to(device)

#         pred = net(graph)
#         ep_v_loss+=loss(pred.float(), label.float())
#     ep_v_loss/=j
    
    epoch_time.append(t_end - t_start)
    if epoch % 2 == 0 and verbose:
        print(f"step #{epoch} | ep_train_loss = {np.sqrt(ep_t_loss.item()):.4f}"
            #f" | ep_valid_loss = {np.sqrt(ep_v_loss):.4f}"
             f" | epoch_time = {t_end - t_start:.2f}"
             f" | lost_batches = {nan_batches}")

step #0 | ep_train_loss = 2.3356 | epoch_time = 6.68 | lost_batches = 0
step #2 | ep_train_loss = 1.2176 | epoch_time = 5.75 | lost_batches = 0
step #4 | ep_train_loss = 2.3946 | epoch_time = 5.88 | lost_batches = 0
step #6 | ep_train_loss = 1.9407 | epoch_time = 6.49 | lost_batches = 0
step #8 | ep_train_loss = 1.0445 | epoch_time = 5.66 | lost_batches = 0
step #10 | ep_train_loss = 1.4131 | epoch_time = 5.69 | lost_batches = 0
step #12 | ep_train_loss = 1.1626 | epoch_time = 5.72 | lost_batches = 0
step #14 | ep_train_loss = 1.0035 | epoch_time = 6.59 | lost_batches = 0
step #16 | ep_train_loss = 1.1313 | epoch_time = 5.82 | lost_batches = 0
step #18 | ep_train_loss = 0.9550 | epoch_time = 5.83 | lost_batches = 0


In [18]:
len(tuple(valid_idxs))

480

In [22]:
j=0
for graph, label, index in validset:
    j+=1

    label = label['glabel'][index]
    graph = graph[0]
    # Transfer to GPU
    graph, label = graph.to(
        device), label.to(device)

    pred = net(graph)
    ep_v_loss+=loss(pred.float(), label.float())
    if j%20==0:
        print(j)
ep_v_loss/=j

20
40
60
80
100
120
140
160
180
200
220
240
260
280
300
320
340


DGLError: [01:37:06] /opt/dgl/src/graph/heterograph.cc:509: Check failed: magicNum == kDGLSerialize_HeteroGraph (15937764024707752255 vs. 15949673719616654015) : Invalid HeteroGraph Data
Stack trace:
  [bt] (0) /home/raul/anaconda3/envs/python/lib/python3.8/site-packages/dgl/libdgl.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x4f) [0x7f8ce08827af]
  [bt] (1) /home/raul/anaconda3/envs/python/lib/python3.8/site-packages/dgl/libdgl.so(dgl::HeteroGraph::Load(dmlc::Stream*)+0x151) [0x7f8ce0f97981]
  [bt] (2) /home/raul/anaconda3/envs/python/lib/python3.8/site-packages/dgl/libdgl.so(dgl::serialize::LoadHeteroGraphs(std::string const&, std::vector<unsigned long, std::allocator<unsigned long> >)+0x7ea) [0x7f8ce101d51a]
  [bt] (3) /home/raul/anaconda3/envs/python/lib/python3.8/site-packages/dgl/libdgl.so(+0x99a3df) [0x7f8ce10163df]
  [bt] (4) /home/raul/anaconda3/envs/python/lib/python3.8/site-packages/dgl/libdgl.so(DGLFuncCall+0x48) [0x7f8ce0f34478]
  [bt] (5) /home/raul/anaconda3/envs/python